# 2. Finetune VGG-16

In [1]:
import keras
from keras.applications.vgg16 import VGG16

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten

/home/dustin/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/dustin/anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import pandas as pd
import numpy as np
import pickle
import os

In [17]:
#pickle_path = 'Pickled/Small_Train/'

pickle_path = 'Pickled/Train_Data/'

#pickle_path = 'Pickled/Test_Data/'

# 2.0 Define function that gets chunks of dataset.

In [4]:
# Returns i-th chunk of X,y data at path.
def Data_Gen(X_path, y_path):
        
    file = open(pickle_path + "Spectra/" + X_path, "rb")
    X=pickle.load(file)
    file.close()
    
    # I'm going to feed this into a pretrained Vgg16 model which expects a 3 channel image input.
    # Requires more memory, but less work. 
    X = np.stack((X,)*3, -1)
    
    
    file = open(pickle_path + "Targets/" + y_path, "rb")
    y=pickle.load(file)
    file.close()
    
    # Convert y to one-hot
    y = keras.utils.np_utils.to_categorical(y)        
    
    return X,y

# 2.1 Define VGG as Base Model

In [5]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(129,129,3))

# 2.2 Add model on top of VGG

In [6]:
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(1024, activation='relu'))
top_model.add(Dense(512, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(6, activation='softmax'))

In [7]:
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

In [8]:
# GPU memory is in short supply on my PC. 
del base_model, top_model

# 2.3 Set only Block5 and Sequential1 to be trainable. 

In [6]:
for layer in model.layers[:]:
    layer.trainable = True

for layer in model.layers[:14]:
    layer.trainable = False

In [7]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 129, 129, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 129, 129, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 129, 129, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

# 2.4 Train model!

### After playing around with a small subset of the training data I found that 4 epochs worked best to minimize overfitting. The training data is split into 260 chunks. I iterate through chunk by chunk, doing for 4 epochs on each. 

In [9]:
X_files = sorted(os.listdir(pickle_path + "Spectra"))
y_files = sorted(os.listdir(pickle_path + "Targets"))

num_chunks = len(y_files)
iteration = 0

for X_file, y_file in zip(X_files,y_files):
    
    iteration +=1
    print('chunk number: ' + str(iteration) + " of " + str(num_chunks))
    
    X,y = Data_Gen(X_file, y_file)
    
    model.fit(X, y, batch_size=32, epochs=4)

chunk number: 1 of 260
Epoch 1/4
1024/1024 [==============================] - 14s 14ms/step - loss: 6.1931 - acc: 0.2402
Epoch 2/4
1024/1024 [==============================] - 10s 10ms/step - loss: 1.5594 - acc: 0.3809
Epoch 3/4
1024/1024 [==============================] - 10s 10ms/step - loss: 1.3258 - acc: 0.5264
Epoch 4/4
1024/1024 [==============================] - 11s 10ms/step - loss: 1.2512 - acc: 0.5420
chunk number: 2 of 260
Epoch 1/4
1024/1024 [==============================] - 10s 10ms/step - loss: 1.2507 - acc: 0.5381
Epoch 2/4
1024/1024 [==============================] - 11s 10ms/step - loss: 1.1797 - acc: 0.5576
Epoch 3/4
1024/1024 [==============================] - 11s 10ms/step - loss: 1.1373 - acc: 0.5879
Epoch 4/4
1024/1024 [==============================] - 11s 11ms/step - loss: 1.1061 - acc: 0.5869
chunk number: 3 of 260
Epoch 1/4
1024/1024 [==============================] - 11s 11ms/step - loss: 1.1224 - acc: 0.5791
Epoch 2/4
1024/1024 [============================

1024/1024 [==============================] - 11s 11ms/step - loss: 0.8421 - acc: 0.7061
Epoch 2/4
1024/1024 [==============================] - 11s 11ms/step - loss: 0.6940 - acc: 0.7402
Epoch 3/4
1024/1024 [==============================] - 11s 11ms/step - loss: 0.6066 - acc: 0.7646
Epoch 4/4
1024/1024 [==============================] - 11s 11ms/step - loss: 0.5256 - acc: 0.8008
chunk number: 41 of 260
Epoch 1/4
1024/1024 [==============================] - 11s 11ms/step - loss: 0.8549 - acc: 0.6992
Epoch 2/4
1024/1024 [==============================] - 11s 11ms/step - loss: 0.6812 - acc: 0.7520
Epoch 3/4
1024/1024 [==============================] - 11s 11ms/step - loss: 0.6067 - acc: 0.7842
Epoch 4/4
1024/1024 [==============================] - 11s 11ms/step - loss: 0.5304 - acc: 0.8047
chunk number: 42 of 260
Epoch 1/4
1024/1024 [==============================] - 11s 11ms/step - loss: 0.8113 - acc: 0.7168
Epoch 2/4
1024/1024 [==============================] - 11s 11ms/step - loss: 0.6

1024/1024 [==============================] - 11s 11ms/step - loss: 0.4695 - acc: 0.8203
chunk number: 60 of 260
Epoch 1/4
1024/1024 [==============================] - 11s 11ms/step - loss: 0.8146 - acc: 0.7129
Epoch 2/4
1024/1024 [==============================] - 11s 11ms/step - loss: 0.6255 - acc: 0.7598
Epoch 3/4
1024/1024 [==============================] - 11s 11ms/step - loss: 0.5644 - acc: 0.7910
Epoch 4/4
1024/1024 [==============================] - 11s 11ms/step - loss: 0.4842 - acc: 0.8027
chunk number: 61 of 260
Epoch 1/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.8220 - acc: 0.7061
Epoch 2/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0.6302 - acc: 0.7578
Epoch 3/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.5612 - acc: 0.7793
Epoch 4/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0.4699 - acc: 0.8320
chunk number: 62 of 260
Epoch 1/4
1024/1024 [==============================] - 1

Epoch 1/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0.8321 - acc: 0.7070
Epoch 2/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0.6362 - acc: 0.7686
Epoch 3/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0.5188 - acc: 0.8057
Epoch 4/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0.4451 - acc: 0.8398
chunk number: 100 of 260
Epoch 1/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.8210 - acc: 0.7188
Epoch 2/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0.5957 - acc: 0.7812
Epoch 3/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0.4972 - acc: 0.8164
Epoch 4/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.4143 - acc: 0.8516
chunk number: 101 of 260
Epoch 1/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.8343 - acc: 0.7197
Epoch 2/4
1024/1024 [==============================] - 11s 10ms/step

1024/1024 [==============================] - 11s 10ms/step - loss: 0.6765 - acc: 0.7607
Epoch 2/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.4833 - acc: 0.8203
Epoch 3/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.3624 - acc: 0.8701
Epoch 4/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.2914 - acc: 0.8926
chunk number: 139 of 260
Epoch 1/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0.8687 - acc: 0.7139
Epoch 2/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0.6003 - acc: 0.7793
Epoch 3/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0.4962 - acc: 0.8076
Epoch 4/4
1024/1024 [==============================] - 11s 11ms/step - loss: 0.3887 - acc: 0.8584
chunk number: 140 of 260
Epoch 1/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0.8672 - acc: 0.7012
Epoch 2/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0

1024/1024 [==============================] - 10s 10ms/step - loss: 0.7816 - acc: 0.7500
Epoch 2/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.5617 - acc: 0.8027
Epoch 3/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.4330 - acc: 0.8574
Epoch 4/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.3333 - acc: 0.8955
chunk number: 178 of 260
Epoch 1/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.7758 - acc: 0.7314
Epoch 2/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.5202 - acc: 0.8086
Epoch 3/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.4012 - acc: 0.8506
Epoch 4/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.3056 - acc: 0.8848
chunk number: 179 of 260
Epoch 1/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.7675 - acc: 0.7539
Epoch 2/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0

1024/1024 [==============================] - 10s 10ms/step - loss: 0.8617 - acc: 0.7295
Epoch 2/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.5995 - acc: 0.7910
Epoch 3/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.4728 - acc: 0.8418
Epoch 4/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0.3649 - acc: 0.8701
chunk number: 217 of 260
Epoch 1/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0.8106 - acc: 0.7422
Epoch 2/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.5582 - acc: 0.8027
Epoch 3/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.4124 - acc: 0.8604
Epoch 4/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.3218 - acc: 0.8984
chunk number: 218 of 260
Epoch 1/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.7807 - acc: 0.7207
Epoch 2/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0

1024/1024 [==============================] - 10s 10ms/step - loss: 0.7729 - acc: 0.7246
Epoch 2/4
1024/1024 [==============================] - 10s 10ms/step - loss: 0.5360 - acc: 0.7979
Epoch 3/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0.4379 - acc: 0.8320
Epoch 4/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0.3687 - acc: 0.8643
chunk number: 256 of 260
Epoch 1/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0.8093 - acc: 0.7256
Epoch 2/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0.5567 - acc: 0.7969
Epoch 3/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0.4320 - acc: 0.8457
Epoch 4/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0.3574 - acc: 0.8779
chunk number: 257 of 260
Epoch 1/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0.7695 - acc: 0.7393
Epoch 2/4
1024/1024 [==============================] - 11s 10ms/step - loss: 0

### The model is saved for analysis or further training. 

In [10]:
model.save("Model/Wingbeats_Cat_X_Entropy_V3.h5")

### The next notebook, Evaluate.ipynb, evaluates the performance of this model.